In [1]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")

# This is new
batch["labels"] = torch.tensor([1, 1])

optimizer = AdamW(model.parameters())
loss = model(**batch).loss
loss.backward()
optimizer.step()

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncase

In [2]:
# carichiamo dataset mrpc dal benchmark glue

from datasets import load_dataset

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating test split: 100%|██████████| 1725/1725 [00:00<00:00, 1169604.66 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [7]:
#abbiamo il raw dataset
# coppie di frasi con labels: sinonimi o no

raw_train_dataset = raw_datasets["train"]
raw_train_dataset[15]

{'sentence1': 'Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .',
 'sentence2': 'Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .',
 'label': 0,
 'idx': 16}

In [6]:
#per sapere cosa è 1 e cosa è 0
raw_train_dataset.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

In [13]:
# vanno tokenizzate come coppie. per questo esiste il token_type_ids

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
print(raw_datasets["train"][15]["sentence1"])
print(raw_datasets["train"][15]["sentence2"])

#tokenizzato come coppia
tokenized_sentences_1_2 = tokenizer(raw_datasets["train"][15]["sentence1"],raw_datasets["train"][15]["sentence2"])
print(tokenized_sentences_1_2)

# tokenier senza coppia
tokenized_sentences_1 = tokenizer(raw_datasets["train"][15]["sentence1"])
print(tokenized_sentences_1)


Rudder was most recently senior vice president for the Developer & Platform Evangelism Business .
Senior Vice President Eric Rudder , formerly head of the Developer and Platform Evangelism unit , will lead the new entity .
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102, 3026, 3580, 2343, 4388, 24049, 1010, 3839, 2132, 1997, 1996, 9722, 1998, 4132, 9340, 12439, 2964, 3131, 1010, 2097, 2599, 1996, 2047, 9178, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [101, 24049, 2001, 2087, 3728, 3026, 3580, 2343, 2005, 1996, 9722, 1004, 4132, 9340, 12439, 2964, 2449, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs

# questa cosa che succede dipende dal modello e di come è stato preaddestrato

{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
# ok, tokenizziamo tutto il dataset

tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)
tokenized_dataset[0]

# questo funziona bene ma è troppo grande. salva tutto insieme e con tute le keys()


Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [27]:
#questo metodo serve a tokenizzare uno o un tot di dati alla volta.
# .map serve ad applicare la funzione tekenize ad ogni elemento passato
# non si hanno problemi di memoria e vi è un parallelismo interno.
# se si chiama un batch troppo piccolo (=1) non si guadagna niente

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets





Map: 100%|██████████| 408/408 [00:00<00:00, 33165.55 examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [28]:

# Il Padding è dinamico, è quando viene effettuato con le lunghezza max del batch e non di 
# tutto il dataset. meno uso di memoria e di tempo.
# non funziona con tutte gli hardware, ad esempio la TPU vuole il padding fisso.
# chi si occupa di fare questpo è il DataCollator

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# guardaimo ad esempio le lunghesse dei primi 8 salmples, e vediamo che il max è 67, ù
# quindi si può paddare un batch dei primi 8 a 67 e non maxlenght
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence1", "sentence2"]}
[len(x) for x in samples["input_ids"]]


[50, 59, 47, 67, 59, 50, 62, 32]

In [47]:
from datasets import load_dataset
from transformers import AutoTokenizer

# in questo modo tutti hanno lo stessa lunghezza, 128

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
print(tokenized_dataset.column_names)
print(tokenized_dataset["train"][0]["sentence1"])
print(len(tokenized_dataset["train"][1]["input_ids"]))
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence1","sentence2"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 1725/1725 [00:00<00:00, 27157.53 examples/s]

{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask']}
Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
128


dict_keys(['train', 'validation', 'test'])

In [6]:

# creaimo il padding dinamico
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding

# in questo modo tutti hanno lo stessa lunghezza, 128

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il padding dinamico

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence1","sentence2"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

# sposto il padding a dopo
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True,collate_fn=data_collator)

for step, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step > 5:
        break


torch.Size([16, 76])
torch.Size([16, 67])
torch.Size([16, 79])
torch.Size([16, 74])
torch.Size([16, 71])
torch.Size([16, 73])
torch.Size([16, 74])


In [10]:
# uguale per sst2 dataset
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding

# in questo modo tutti hanno lo stessa lunghezza, 128

raw_datasets = load_dataset("glue", "sst2")
print(raw_datasets.column_names)
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il padding dinamico

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

# sposto il padding a dopo
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True,collate_fn=data_collator)

for step, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step > 5:
        break

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train': ['sentence', 'label', 'idx'], 'validation': ['sentence', 'label', 'idx'], 'test': ['sentence', 'label', 'idx']}


Map: 100%|██████████| 872/872 [00:00<00:00, 67641.31 examples/s]

torch.Size([16, 38])
torch.Size([16, 45])
torch.Size([16, 40])
torch.Size([16, 43])
torch.Size([16, 22])
torch.Size([16, 26])
torch.Size([16, 34])


In [1]:
# uguale per sst2 dataset
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding


raw_datasets = load_dataset("glue", "sst2")
print(raw_datasets.column_names)
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il padding dinamico

def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_function,batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

# sposto il padding a dopo
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=16, shuffle=True,collate_fn=data_collator)

for step, batch in enumerate(train_dataloader):
    print(batch["input_ids"].shape)
    if step > 5:
        break

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'train': ['sentence', 'label', 'idx'], 'validation': ['sentence', 'label', 'idx'], 'test': ['sentence', 'label', 'idx']}


/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 1821/1821 [00:00<00:00, 71435.64 examples/s]

torch.Size([16, 28])
torch.Size([16, 38])
torch.Size([16, 44])
torch.Size([16, 22])
torch.Size([16, 26])
torch.Size([16, 23])
torch.Size([16, 37])


## THE TRAINER

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")   # scarico il dataset
checkpoint = "bert-base-uncased"     
tokenizer = AutoTokenizer.from_pretrained(checkpoint)   # tokenizer del modello pre-trainato


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)   # funzione per tokenizare (utile per batch)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # mappa tokenize function per ogni elemento del dataset (o batch)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il  padding dinamico

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 408/408 [00:00<00:00, 31336.89 examples/s]


In [2]:
# definisco il trainingArguments che contiene gli hyperparametri del trainer
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer") # cartella dove verrà salvato il modello allenato

# definisco il modello
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# Notare che BERT has not been pretrained on classifying pairs of sentences, in automitico 
# cambia l'head in uno adatto e ti avvisa che alcuni pesi non sono settati e
# ti invita a trainare il modello. quello che faremo

# definisco il trainer
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator, # questo non servirebbe perchè è di default 
    tokenizer=tokenizer,
)

# lancio il train (fine-tuning)
#trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_18942/3061554017.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Evaluation
Inseriamo le metriche di valutazione

In [12]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape) # estraggo le predizioni

import numpy as np
preds = np.argmax(predictions.predictions, axis=-1) #calcolo le labels

import evaluate
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids) # confronto con le labels del dataset e uso le metriche de
#finite nel dataset per valutare


FileNotFoundError: Couldn't find a module script at /home/apetrella/Workspace/tutorial/hugging-face/glue/glue.py. Module 'glue' doesn't exist on the Hugging Face Hub either.

In [1]:
import evaluate
import numpy as np
from transformers import Trainer,TrainingArguments,AutoModelForSequenceClassification
# unisco tutto
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")   # scarico il dataset
checkpoint = "bert-base-uncased"     
tokenizer = AutoTokenizer.from_pretrained(checkpoint)   # tokenizer del modello pre-trainato


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)   # funzione per tokenizare (utile per batch)

#definisco il trainingArguments che contiene gli hyperparametri del trainer

training_args = TrainingArguments("test-trainer") # cartella dove verrà salvato il modello allenato

# definisco il modello
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# Notare che BERT has not been pretrained on classifying pairs of sentences, in automitico 
# cambia l'head in uno adatto e ti avvisa che alcuni pesi non sono settati e
# ti invita a trainare il modello. quello che faremo

# definisco il trainer
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # mappa tokenize function per ogni elemento del dataset (o batch)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il  padding dinamico

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")      # selezione metriche predefinite del dataset (f1 e accuracy)
    logits, labels = eval_preds                 # da evalpreds estraggo i logits predetti e labels corrette
    predictions = np.argmax(logits, axis=-1)    # da logits a label 
                                                # prende il valore massimo (tra 2) di ogni elemento, e restituisce l'indice
                                                # axis=-1 per lavorare riga per riga 
    return metric.compute(predictions=predictions, references=labels) # ritorno il calcolo delle metriche


training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # passo la funzione al trainer
)

trainer.train()

/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.628867,0.696078,0.818182
2,0.628100,0.415041,0.791667,0.831014
3,0.477600,0.458526,0.852941,0.896194


TrainOutput(global_step=1377, training_loss=0.4930462473617919, metrics={'train_runtime': 60.8971, 'train_samples_per_second': 180.698, 'train_steps_per_second': 22.612, 'total_flos': 405114969714960.0, 'train_loss': 0.4930462473617919, 'epoch': 3.0})

In [3]:
# ✏️ Try it out! Fine-tune a model on the GLUE SST-2 dataset, using the data processing you did in section 2.

# Sentiment analys positivo/negativo

import evaluate
import numpy as np
from transformers import Trainer,TrainingArguments,AutoModelForSequenceClassification
# unisco tutto
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "sst2")   # scarico il dataset
checkpoint = "bert-base-uncased"     
tokenizer = AutoTokenizer.from_pretrained(checkpoint)   # tokenizer del modello pre-trainato


def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)   # funzione per tokenizare (utile per batch)

#definisco il trainingArguments che contiene gli hyperparametri del trainer

training_args = TrainingArguments("test-trainer") # cartella dove verrà salvato il modello allenato

# definisco il modello
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
# Notare che BERT has not been pretrained on classifying pairs of sentences, in automitico 
# cambia l'head in uno adatto e ti avvisa che alcuni pesi non sono settati e
# ti invita a trainare il modello. quello che faremo

# definisco il trainer
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True) # mappa tokenize function per ogni elemento del dataset (o batch)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)  # per il  padding dinamico

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "sst2")      # selezione metriche predefinite del dataset (f1 e accuracy)
    logits, labels = eval_preds                 # da evalpreds estraggo i logits predetti e labels corrette
    predictions = np.argmax(logits, axis=-1)    # da logits a label 
                                                # prende il valore massimo (tra 2) di ogni elemento, e restituisce l'indice
                                                # axis=-1 per lavorare riga per riga 
    return metric.compute(predictions=predictions, references=labels) # ritorno il calcolo delle metriche


training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, # passo la funzione al trainer
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1821/1821 [00:00<00:00, 76009.63 examples/s]
/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_28380/2001996043.py:46: FutureWarning: `tokenizer` is deprecated and will be rem

Epoch,Training Loss,Validation Loss,Accuracy
1,0.254200,0.480284,0.886468
2,0.164500,0.474670,0.892202
3,0.104400,0.450810,0.904817


TrainOutput(global_step=25257, training_loss=0.19976031708973163, metrics={'train_runtime': 799.0799, 'train_samples_per_second': 252.85, 'train_steps_per_second': 31.608, 'total_flos': 3082513027395900.0, 'train_loss': 0.19976031708973163, 'epoch': 3.0})

### Facciamo tutto senza usare la Classe Trainer!!!!


In [ ]:
# PREPROCESS DATA
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader
import torch
from tqdm.auto import tqdm

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ELIMIARE LE COLONNE (e altro )NEL DATASET CHE NON CI INTERESSANO (La classe trainer lo fa in automatico)

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
print("colonne rimaste nel dataset:")
print(tokenized_datasets["train"].column_names)

# DEFINIAMO IL DATALOADER

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)
for batch in train_dataloader: # CHECK CORRECTENESS OF DATA
    break
{k: v.shape for k, v in batch.items()}


# INSTANZIO IL MODELLO

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
outputs = model(**batch)   # CONTROLLLO CHE TUTTO È OK passando il batch (batch da 8, 2 logits)
print(outputs.loss, outputs.logits.shape)


            #### CREIAMO IL TRAINING ####

# OTTIMIZZATORE E LR SCHEDULER, COME QUELLI DEL TRAINING USATO SOPRA
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

# SCELGO LA gpu COME DEVICE
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(device)

# THE LOOP TRAINING
progress_bar = tqdm(range(num_training_steps))

model.train()   # SETTA IL MODELLO PER ESSERE TRAINATO
for epoch in range(num_epochs):   # TUTTO IL DATALOADER PER TOT EPOCHE
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

# THE EVALUAZTION LOOP 
metric = evaluate.load("glue", "mrpc") # QUELLE STANDARD PER QUEL TASK
model.eval() # SETTA IL MODELLO PER FARE INFERENZA (NO DROPOUT / BATCH NORMALIZATION)
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


colonne rimaste nel dataset:
['labels', 'input_ids', 'token_type_ids', 'attention_mask']


/home/apetrella/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


tensor(0.6993, grad_fn=<NllLossBackward0>) torch.Size([8, 2])
1377
cuda


100%|██████████| 1377/1377 [00:37<00:00, 38.48it/s]

{'accuracy': 0.8480392156862745, 'f1': 0.8923611111111112}

100%|██████████| 1377/1377 [00:49<00:00, 38.48it/s]

In [13]:
# USARE ACCELERATORS-- non funzia ma ok per ora.. 
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler
from torch.utils.data import DataLoader
import torch
from tqdm.auto import tqdm
from accelerate import Accelerator
from accelerate import notebook_launcher

def training_function():
    raw_datasets = load_dataset("glue", "mrpc")
    checkpoint = "bert-base-uncased"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)

    def tokenize_function(example):
        return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    # ELIMIARE LE COLONNE (e altro )NEL DATASET CHE NON CI INTERESSANO (La classe trainer lo fa in automatico)

    tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets.set_format("torch")
    #print("colonne rimaste nel dataset:")
    #print(tokenized_datasets["train"].column_names)

    # DEFINIAMO IL DATALOADER

    train_dataloader = DataLoader(
        tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
    )
    eval_dataloader = DataLoader(
        tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
    )
    #for batch in train_dataloader: # CHECK CORRECTENESS OF DATA
    #    break
    #{k: v.shape for k, v in batch.items()}


    # instanzion accelerate
    accelerator = Accelerator()

    # INSTANZIO IL MODELLO

    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    outputs = model(**batch)   # CONTROLLLO CHE TUTTO È OK passando il batch (batch da 8, 2 logits)
    #print(outputs.loss, outputs.logits.shape)


                #### CREIAMO IL TRAINING ####

    # OTTIMIZZATORE E LR SCHEDULER, COME QUELLI DEL TRAINING USATO SOPRA
    optimizer = AdamW(model.parameters(), lr=5e-5)


    # SCELGO LA gpu COME DEVICE  (rimuovo)
    #device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    # model.to(device)
    # print(device)

    train_dataloader, eval_dataloader, model, optimizer = accelerator.prepare(
        train_dataloader, eval_dataloader, model, optimizer
    )


    num_epochs = 3
    num_training_steps = num_epochs * len(train_dataloader)
    lr_scheduler = get_scheduler(
        "linear",
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps,
    )
    #print(num_training_steps)


    # THE LOOP TRAINING
    progress_bar = tqdm(range(num_training_steps))

    model.train()   # SETTA IL MODELLO PER ESSERE TRAINATO
    for epoch in range(num_epochs):   # TUTTO IL DATALOADER PER TOT EPOCHE
        for batch in train_dataloader:
            # batch = {k: v.to(device) for k, v in batch.items()}   # rimuovo
            outputs = model(**batch)
            loss = outputs.loss
            # loss.backward() # rimuovo
            accelerator.backward(loss) # (aggiungo)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)

    # THE EVALUAZTION LOOP 
    metric = evaluate.load("glue", "mrpc") # QUELLE STANDARD PER QUEL TASK
    model.eval() # SETTA IL MODELLO PER FARE INFERENZA (NO DROPOUT / BATCH NORMALIZATION)
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    metric.compute()

notebook_launcher(training_function)

ValueError: You have to specify the number of GPUs you would like to use, add `num_processes=...` to your call.